In [1]:
import string
import itertools
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

In [4]:
import mdtraj as md

In [5]:
with open("cullpdb_pc15.0_res0.0-1.5_len40-10000_R0.25_Xray_d2024_03_18_chains1864.txt", 'r') as f:
    raw = f.readlines()
pdbids = [r.split()[0] for r in raw][1:]
print(pdbids)

['5D8VA', '3NIRA', '5NW3A', '1UCSA', '3X2MA', '2VB1A', '1US0A', '6E6OA', '6S2MA', '1R6JA', '4REKA', '4I8HA', '2WFIA', '2OV0A', '2B97A', '8C5NA', '3X34A', '7KR0A', '6L27A', '5YCEA', '1GCIA', '7A5MA', '5GV8A', '6ZM8A', '1X6ZA', '4UA6A', '5TDAA', '7AF2AAA', '3UI4A', '5AL6A', '5NFMA', '1W0NA', '5HB7A', '2VXNA', '1NWZA', '8C3XA', '7AVKA', '2JFRA', '2O9SA', '4Y9WA', '6TGUA', '1P9GA', '7BNHA', '2O7AA', '2YKZA', '4EICA', '6EIOA', '4PSSA', '4AYOA', '6Q00A', '6Q00B', '3O4PA', '6ETLA', '1MC2A', '1X8QA', '7BBXA', '7AOTA', '3QPAA', '2FMAA', '5XVTA', '2F01A', '1G6XA', '3WDNA', '1MUWA', '3ZSJA', '6ZPAA', '2DDXA', '1V6PA', '8EREA', '4HS1A', '5L87A', '1GWEA', '3ZOJA', '3FILA', '4F1VA', '5LP9A', '1I1WA', '4U9HL', '4U9HS', '6KFNA', '7TVLA', '4O6UA', '4UYRA', '3IP0A', '8AVUA', '4WEEA', '4Y9VA', '1IX9A', '6KLZA', '7OUZA', '6FMCA', '4NPDA', '1VYRA', '5X9LA', '7PSYA', '3VORA', '3G21A', '5GJIA', '3ZR8X', '4EA9A', '2XU3A', '6LK1A', '1J0PA', '5IG6A', '5BR4A', '3X0IA', '8CR4A', '5MK9A', '4HNOA', '1IQZA', '6EQEA'

In [6]:
import Bio
from Bio.PDB import PDBList

pdbl = PDBList()
for prot in pdbids:
    pdbl.retrieve_pdb_file(prot[:4], file_format='pdb')

Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/d8/pdb5d8v.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/ni/pdb3nir.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/nw/pdb5nw3.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/uc/pdb1ucs.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/x2/pdb3x2m.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/vb/pdb2vb1.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/us/pdb1us0.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/e6/pdb6e6o.ent' 
Structure exists: '/Users/daviddesancho/Research/Projects/IDPs/LLPS/Stickers/PDBanalysis/s2/pdb6s2m.ent' 
Structure exists: '/Users/daviddesancho/Resear

In [40]:
!mv */pdb*ent PDB/

mv: PDB/pdb1g6x.ent and PDB/pdb1g6x.ent are identical
mv: PDB/pdb1gci.ent and PDB/pdb1gci.ent are identical
mv: PDB/pdb1gwe.ent and PDB/pdb1gwe.ent are identical
mv: PDB/pdb1i1w.ent and PDB/pdb1i1w.ent are identical
mv: PDB/pdb1ix9.ent and PDB/pdb1ix9.ent are identical
mv: PDB/pdb1mc2.ent and PDB/pdb1mc2.ent are identical
mv: PDB/pdb1muw.ent and PDB/pdb1muw.ent are identical
mv: PDB/pdb1nwz.ent and PDB/pdb1nwz.ent are identical
mv: PDB/pdb1p9g.ent and PDB/pdb1p9g.ent are identical
mv: PDB/pdb1r6j.ent and PDB/pdb1r6j.ent are identical
mv: PDB/pdb1ucs.ent and PDB/pdb1ucs.ent are identical
mv: PDB/pdb1us0.ent and PDB/pdb1us0.ent are identical
mv: PDB/pdb1v6p.ent and PDB/pdb1v6p.ent are identical
mv: PDB/pdb1vyr.ent and PDB/pdb1vyr.ent are identical
mv: PDB/pdb1w0n.ent and PDB/pdb1w0n.ent are identical
mv: PDB/pdb1x6z.ent and PDB/pdb1x6z.ent are identical
mv: PDB/pdb1x8q.ent and PDB/pdb1x8q.ent are identical
mv: PDB/pdb2b97.ent and PDB/pdb2b97.ent are identical
mv: PDB/pdb2ddx.ent and PDB/

In [49]:
dPHE = []; dTYR = []
dPHE_ca = []; dTYR_ca = []
for prot in pdbids:
    print (prot[:4], prot[4])
    try:
        pdb_file = md.load_pdb("PDB/pdb%s.ent"%prot.lower()[:4])
    except FileNotFoundError as e:
        print (e)
        continue

    chainid = string.ascii_uppercase.index(prot[4])
    try:
        sel = pdb_file.top.select('chainid %i'%chainid)
        pdb_file = pdb_file.atom_slice(sel)
    except IndexError as e:
        print (e)
        continue
        
    PHE = [x.index for x in pdb_file.topology.residues if x.name == 'PHE']
    
    try:
        contacts = md.compute_contacts(pdb_file, contacts=list(itertools.combinations(PHE, 2)))
        dPHE += contacts[0].ravel().tolist()
        contacts = md.compute_contacts(pdb_file, contacts=list(itertools.combinations(PHE, 2)), scheme='ca')
        dPHE_ca += contacts[0].ravel().tolist()
    except ValueError as e:
        print (e)
        print (" Not enough Phe residues in protein", PHE)

    TYR = [x.index for x in pdb_file.topology.residues if x.name == 'TYR']
    try:
        contacts = md.compute_contacts(pdb_file, contacts=list(itertools.combinations(TYR, 2)))
        dTYR += contacts[0].ravel().tolist()
        contacts = md.compute_contacts(pdb_file, contacts=list(itertools.combinations(TYR, 2)), scheme='ca')
        dTYR_ca += contacts[0].ravel().tolist()
    except ValueError as e:
        print (e)
        print (" Not enough Tyr residues in protein", TYR)

5D8V A
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein [18]
3NIR A
contacts must be ndim 2. You supplied 1
 Not enough Phe residues in protein [12]
5NW3 A
1UCS A
contacts must be ndim 2. You supplied 1
 Not enough Phe residues in protein []
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein [62]
3X2M A
2VB1 A
1US0 A
6E6O A
contacts must be ndim 2. You supplied 1
 Not enough Phe residues in protein []
6S2M A
1R6J A
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein []
4REK A
4I8H A
2WFI A
2OV0 A
2B97 A
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein []
8C5N A
3X34 A
7KR0 A
6L27 A
5YCE A
1GCI A
7A5M A
5GV8 A
6ZM8 A
1X6Z A
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein []
4UA6 A
5TDA A
7AF2 A
[Errno 2] No such file or directory: 'PDB/pdb7af2.ent'
3UI4 A
5AL6 A
contacts must be ndim 2. You supplied 1
 Not enough Tyr residues in protein [41]
5NFM A
contacts 

ValueError: substring not found

In [ ]:
nbins = 100
bins = np.linspace(0.2,2,nbins+1)
dx = bins[1] - bins[0]

fig, ax = plt.subplots()

hist, edges = np.histogram(dPHE, bins=bins, density=True)
centers = np.array([0.5*(edges[i] + edges[i+1]) for i in range(nbins)])
norm = 4*np.pi*centers**2*dx
ax.plot(centers, -np.log(hist/norm), label='F')

hist, edges = np.histogram(dTYR, bins=bins, density=True)
ax.plot(centers, -np.log(hist/norm), label='Y')
ax.set_xlabel('r (nm)')
ax.set_ylabel('ln(g(r))')
plt.legend()

In [ ]:
nbins = 100
bins = np.linspace(0.3,2,nbins+1)
dx = bins[1] - bins[0]

fig, ax = plt.subplots()

hist, edges = np.histogram(dPHE_ca, bins=bins, density=True)
centers = np.array([0.5*(edges[i] + edges[i+1]) for i in range(nbins)])
norm = 4*np.pi*centers**2*dx
ax.plot(centers, -np.log(hist/norm), label='F')

hist, edges = np.histogram(dTYR_ca, bins=bins, density=True)
ax.plot(centers, -np.log(hist/norm), label='Y')
ax.set_xlabel('r$_{C^\alpha}$(nm)')
ax.set_ylabel('ln(g(r))')
plt.legend()